In [52]:
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint

In [2]:
structures = pd.read_feather('../data/structures.feather')
train = pd.read_feather('../data/train.feather')
test = pd.read_feather('../data/test.feather')

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [15]:
methane_name = 'dsgdb9nsd_000001'
methane = structures[structures.molecule_name == methane_name]

ethanol_name = 'dsgdb9nsd_000014'
ethanol = structures[structures.molecule_name == ethanol_name]

In [16]:
methane

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [18]:
from ase import Atoms

In [54]:
def get_dist(atom1, atom2):
    dx = atom1.x - atom2.x
    dy = atom1.y - atom2.y
    dz = atom1.z - atom2.z
    diffsq = dx * dx + dy * dy + dz * dz
    return np.sqrt(diffsq)

molecule_dist = {
    ('C', 'C'): (1.2, 1.54),
    ('C', 'H'): (1.06, 1.12),
    ('C', 'O'): (1.43, 2.15),
    ('C', 'N'): (1.47, 2.10),
    ('C', 'F'): (1.34, 1.34),
}

def get_bond_size(atom1, atom2):
    pair = (atom1.atom, atom2.atom)
    if pair in molecule_dist:
        return molecule_dist[pair]
    
    pair = (atom2.atom, atom1.atom)
    if pair in molecule_dist:
        return molecule_dist[pair]
    
    return (np.inf, np.inf)

def compute_bonds(molecule_df):
    positions = []
    name = molecule_df.iloc[0, 0]
    bonds = set()
    for i, atom1 in molecule_df.iterrows():
        for j, atom2 in molecule_df.iterrows():
            if i >= j:
                pass
            dist = get_dist(atom1, atom2)
            bond_len = get_bond_size(atom1, atom2)
            if dist >= bond_len[0] and dist <= bond_len[1]:
                bond = ((atom1.atom, atom1.atom_index), (atom2.atom, atom2.atom_index))
                if bond not in bonds and (bond[1], bond[0]) not in bonds:
                    bonds.add(bond)
                
    return bonds

pprint(compute_bonds(ethanol))

{(('C', 0), ('C', 1)),
 (('C', 0), ('H', 3)),
 (('C', 0), ('H', 4)),
 (('C', 0), ('H', 5)),
 (('C', 1), ('H', 6)),
 (('C', 1), ('H', 7))}


In [50]:
[1, 2, 3].reverse()